In [1]:
import os
from pathlib import Path

from tqdm import tqdm

import torch
from torch.utils.data import DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import torch.nn as nn 
import torch.optim as optim
from torchinfo import summary

import config
import dataset
import models
import loss
import val_epoch
import utils

# Validation Dataset

In [2]:
# VALIDATION DATASET
val_transform = A.Compose([
    A.Resize(config.IMG_H, config.IMG_W, p=1),
    ToTensorV2(p=1),
    ]
)

print("\nTEST DFire dataset")
val_dataset = dataset.DFireDataset(
    img_h = config.IMG_H,
    img_w = config.IMG_W,
    img_dir = config.VAL_IMG_DIR,
    label_dir = config.VAL_LABEL_DIR,
    num_classes = config.N_CLASSES,
    ds_len = config.DS_LEN,
    transform=val_transform)

print(f'Test dataset len: {len(val_dataset)}')

# LOADERS
val_loader = DataLoader(dataset=val_dataset,
                        batch_size=config.BATCH_SIZE,
                        num_workers=config.NUM_WORKERS,
                        pin_memory=config.PIN_MEMORY,
                        shuffle=False,
                        drop_last=True)


TEST DFire dataset
DFire Removed wrong images: 0
DFire empty images: 2005
DFire only smoke images: 1186
DFire only fire images: 220
DFire smoke and fire images: 895
Test dataset len: 4306


# Loss Function

In [3]:
# LOSS FUNCTION
if config.LOSS_FN == "BCE":
    print(f'Loss Function: BCE')
    print(f'Smoke Precision Weight: {config.SMOKE_PRECISION_WEIGHT}')
    loss_fn = loss.BCE_LOSS(device=config.DEVICE, smoke_precision_weight=config.SMOKE_PRECISION_WEIGHT)
else:
    print("Wrong loss function")
    raise SystemExit("Wrong loss function")

Loss Function: BCE
Smoke Precision Weight: 0.8


# Setup Model and Load Weights

In [4]:
if config.MODEL == "BED":
    print("Using BED Classifier")
    model = models.BED_CLASSIFIER(num_classes=config.N_CLASSES).to(config.DEVICE)  
else:
    print("Wrong Model")
    raise SystemExit("Wrong Model")

Using BED Classifier


In [5]:
optimizer = optim.Adam(
    model.parameters(), 
    lr=config.LEARNING_RATE, 
    weight_decay=config.WEIGHT_DECAY)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, 
    mode='min',
    factor=config.FACTOR, 
    patience=config.PATIENCE, 
    threshold=config.THRES, 
    threshold_mode='abs',
    min_lr=config.MIN_LR)

# Load Model

In [6]:
#model_name = 'BED_classifier__smoke__precision=0.9219__recall=0.9152__epoch=124.pt'
model_name = 'BED_classifier__smoke__precision=0.9123__recall=0.9127__epoch=133.pt'
model_path = 'experiments/v3_checkpoints_&_moreSaves_&_affineFixed/weights/' + model_name
epoch_saved = utils.load_checkpoint(model_path, model, optimizer, scheduler, config.DEVICE)

Loading Model. Trained during 133 epochs


# Fuse Conv2d and BatchNorm

In [7]:
modules_to_fuse = [ 
    ["model.conv1", "model.bn1"],
    ["model.conv2", "model.bn2"],
    ["model.conv31", "model.bn31"],
    ["model.conv32", "model.bn32"],
    ["model.conv33", "model.bn33"],
    ["model.conv34", "model.bn34"],
    ["model.conv41", "model.bn41"],
    ["model.conv42", "model.bn42"],
    ["model.conv43", "model.bn43"],
    ["model.conv44", "model.bn44"],
    ["model.conv45", "model.bn45"],
    ["model.conv46", "model.bn46"]
]

In [8]:
model.eval()
fused_model = torch.ao.quantization.fuse_modules(model, modules_to_fuse)

# Print Fused Model

In [9]:
print(summary(fused_model, input_size=(config.BATCH_SIZE, 3, config.IMG_H, config.IMG_W)))

Layer (type:depth-idx)                   Output Shape              Param #
BED_CLASSIFIER                           [64, 2]                   --
├─Sequential: 1-1                        [64, 2]                   --
│    └─Conv2d: 2-1                       [64, 32, 224, 224]        896
│    └─Identity: 2-2                     [64, 32, 224, 224]        --
│    └─ReLU: 2-3                         [64, 32, 224, 224]        --
│    └─Dropout2d: 2-4                    [64, 32, 224, 224]        --
│    └─MaxPool2d: 2-5                    [64, 32, 112, 112]        --
│    └─Conv2d: 2-6                       [64, 16, 112, 112]        4,624
│    └─Identity: 2-7                     [64, 16, 112, 112]        --
│    └─ReLU: 2-8                         [64, 16, 112, 112]        --
│    └─Dropout2d: 2-9                    [64, 16, 112, 112]        --
│    └─MaxPool2d: 2-10                   [64, 16, 56, 56]          --
│    └─Conv2d: 2-11                      [64, 16, 56, 56]          272
│    └─Ide

# Evaluate Fused Model vs Un-Fused Model

In [10]:
model.eval()
fused_model.eval()

with torch.no_grad():
    print("____________________________ MODEL BEFORE FUSION ____________________________")
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=val_loader, 
        model=model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)
    print("\n____________________________ MODEL AFTER FUSION ____________________________")
    val_losses, val_metrics = val_epoch.eval_fn(
        loader=val_loader, 
        model=fused_model,                         
        loss_fn=loss_fn,
        device=config.DEVICE)

____________________________ MODEL BEFORE FUSION ____________________________


Validating: 100%|███████████████████████████████| 67/67 [00:02<00:00, 24.85it/s]


Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
19.358      |12.744      |6.613       
SMOKE -> Precision: 0.912 - Recall: 0.913 - Accuracy: 0.916 - F1: 0.913
FIRE -> Precision: 0.931 - Recall: 0.923 - Accuracy: 0.962 - F1: 0.927

____________________________ MODEL AFTER FUSION ____________________________


Validating: 100%|███████████████████████████████| 67/67 [00:02<00:00, 26.69it/s]

Total Loss  |Smoke Loss  |Fire Loss   
------------ ------------ ------------
19.357      |12.745      |6.613       
SMOKE -> Precision: 0.912 - Recall: 0.913 - Accuracy: 0.916 - F1: 0.913
FIRE -> Precision: 0.931 - Recall: 0.923 - Accuracy: 0.962 - F1: 0.927


# Save Fused Model

In [11]:
checkpoint_name = 'BED_classifier__fused_ConvBN.pt'
utils.save_checkpoint(epoch_saved, fused_model, optimizer, scheduler, checkpoint_name)

# Some print syntax

In [12]:
model.model.bn1.weight

Parameter containing:
tensor([0.7632, 0.7238, 0.6478, 0.6555, 0.7178, 0.7809, 0.7142, 0.6737, 0.7869,
        0.6782, 0.7620, 0.6976, 0.7583, 0.6998, 0.7008, 0.7907, 0.7894, 0.6743,
        0.7759, 0.7935, 0.7893, 0.7648, 0.8405, 0.7778, 0.7546, 0.6543, 0.5508,
        0.7858, 0.7749, 0.6656, 0.7774, 0.7743], device='cuda:0',
       requires_grad=True)

In [13]:
# Name is of type string
for name, mod in fused_model.named_modules():
    print(name, mod)

 BED_CLASSIFIER(
  (model): Sequential(
    (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn1): Identity()
    (relu1): ReLU()
    (dropout1): Dropout2d(p=0.3, inplace=False)
    (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv2): Conv2d(32, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): Identity()
    (relu2): ReLU()
    (dropout2): Dropout2d(p=0.3, inplace=False)
    (maxpool3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (conv31): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
    (bn31): Identity()
    (relu31): ReLU()
    (conv32): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn32): Identity()
    (relu32): ReLU()
    (conv33): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
    (bn33): Identity()
    (relu33): ReLU()
    (conv34): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn34): Identity(